# Training of base Neural-PI controllers

In [ ]:
# A RNN-based Reinforcement Learning Framework for Frequency Control Problem with Stability Guarantee
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import RNN
import tensorflow.keras.backend as K
import copy
from mat4py import loadmat
from tensorflow.keras import layers
from tensorflow import keras
import networkx as nx
import scipy
import pickle
import time
from cycler import cycler
import matplotlib as mpl
from matplotlib import ticker

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
# Frequency Control Porblem Environment
class Frequency():
    def  __init__(self, Pm,M,current_M,D,F,delta_t,max_action,dim_action,Penalty_action, coef_cost,  incidence_communication,gamma=5):
        self.param_gamma = gamma
        self.M = current_M
        self.D = D
        self.F = F
        self.Pm = Pm
        self.original_M = M
        self.max_action = max_action
        self.dim_action = dim_action
        self.omega_scale = 2*np.pi# this change the unit of omega to Hz
        self.state_x = []
        self.delta_t = delta_t
        self.Penalty_action = Penalty_action
        self.state_x_transfer1 = np.vstack((np.hstack((np.identity(dim_action,dtype = np.float32),np.zeros((dim_action,dim_action),dtype = np.float32))),\
                                np.hstack((delta_t*self.omega_scale*np.identity(dim_action,dtype = np.float32),\
                                           np.identity(dim_action,dtype = np.float32)-delta_t*np.diag(D/self.M)))))
        self.state_x_transferF = -delta_t*(((self.M**(-1)).reshape(dim_action,1))@np.ones((1,dim_action),dtype = np.float32))*F
        self.state_x_transfer2 = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),
                                            np.identity(dim_action,dtype = np.float32)))        

        self.state_x_transfer3 = np.hstack((np.zeros((1,dim_action),dtype = np.float32),
                                            delta_t*Pm*(self.M**(-1))))
        self.state_x_transfer3_Pm = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),
                                               delta_t*np.diag((self.M**(-1)))))
        self.state_x_transfer4 = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),
                                            -delta_t*np.diag((self.M**(-1)))))

        self.select_add_w = np.vstack((np.zeros((dim_action,1),dtype = np.float32),\
                                        np.ones((dim_action,1),dtype = np.float32)))
        self.select_w = np.vstack((np.zeros((dim_action,dim_action),dtype = np.float32),\
                                        np.identity(dim_action,dtype = np.float32)))
        self.select_delta = np.vstack((np.identity(dim_action,dtype = np.float32),\
                                        np.zeros((dim_action,dim_action),dtype = np.float32)))
        self.diag_c  =  np.diag(coef_cost)
        self.incidence_communication = incidence_communication.astype(np.float32)
        self.last_action = 0

        

    def step_PI_CommEdge(self, actionP, actionI,Pm):

        cost_action, grad_action  =  self.calc_grad_action(actionI)
        action  =  actionP + actionI
        dot_s_new  =  self.state_x@self.select_w-self.CommEdgeFeedback(grad_action@self.incidence_communication)@\
                      self.incidence_communication.T@self.diag_c*0.05
        self.state_x  =  copy.deepcopy(self.state_x@self.state_x_transfer1\
              + np.sum(np.sin( np.transpose(self.state_x@self.select_delta)@np.ones((1,self.dim_action),dtype = np.float32)-\
                np.ones((self.dim_action,1),dtype = np.float32)@(self.state_x@self.select_delta))*self.state_x_transferF,axis = 1 )\
                      @self.state_x_transfer2\
              + Pm@self.state_x_transfer3_Pm\
                          +action@self.state_x_transfer4)                
        self.state_s  =  self.state_s+self.delta_t*dot_s_new/1
        loss_action = 0.5*np.sum(np.power(action,2)@self.diag_c)
        loss = 1000*np.linalg.norm(self.state_x@self.select_add_w,2)+1000*np.max(np.abs(self.state_x@self.select_add_w))+loss_action
        return self.state_s, self.state_x, loss  

    def step_PI_CommEdge_WoCost(self, actionP, actionI,Pm):

        action  =  actionP + actionI
        dot_s_new  =  self.state_x@self.select_w
        self.state_x  = copy.deepcopy(self.state_x@self.state_x_transfer1\
              + np.sum(np.sin( np.transpose(self.state_x@self.select_delta)@np.ones((1,self.dim_action),dtype = np.float32)-\
                np.ones((self.dim_action,1),dtype = np.float32)@(self.state_x@self.select_delta))*self.state_x_transferF,axis = 1 )\
                      @self.state_x_transfer2\
              + Pm@self.state_x_transfer3_Pm\
                          +action@self.state_x_transfer4)              
        self.state_s  =  self.state_s+self.delta_t*dot_s_new/1
        loss  =  self.param_gamma*pow(self.state_x,2)@self.select_add_w 
        # loss_action = 0.5*np.sum(np.power(action,4)@self.diag_c) + 0.05*np.sum(np.power(action,2)@self.diag_c)
        loss_action = 0.5*np.sum(np.power(action,2)@self.diag_c)
        loss = loss + 3*loss_action
        return self.state_s, self.state_x, loss  

    def CommEdgeFeedback(self, x, func = 'linear1'):   
        if func == 'linear1':
            y = x
        return y

    def calc_grad_action(self, action):
        # return 0.5*(action**4)@self.diag_c + 0.05*(action**2)@self.diag_c, 2*(action**3)@self.diag_c #+  0.1*action@self.diag_c
        return 0.5*(action**2)@self.diag_c, action@self.diag_c

    def set_state(self, state_input):
        self.state_x = state_input
        self.state_s  =  np.zeros((1,self.dim_action),dtype = np.float32)
    
    def switch(self,new_percentage):
        self.M = new_percentage*self.original_M
        self.state_x_transfer1 = np.vstack((np.hstack((np.identity(self.dim_action,dtype = np.float32),np.zeros((self.dim_action,self.dim_action),dtype = np.float32))),\
                                np.hstack((self.delta_t*self.omega_scale*np.identity(self.dim_action,dtype = np.float32),\
                                           np.identity(self.dim_action,dtype = np.float32)-self.delta_t*np.diag(self.D/self.M)))))
        self.state_x_transferF = -self.delta_t*(((self.M**(-1)).reshape(self.dim_action,1))@np.ones((1,self.dim_action),dtype = np.float32))*self.F   

        self.state_x_transfer3 = np.hstack((np.zeros((1,self.dim_action),dtype = np.float32),
                                            self.delta_t*self.Pm*(self.M**(-1))))
        self.state_x_transfer3_Pm = np.hstack((np.zeros((self.dim_action,self.dim_action),dtype = np.float32),
                                               self.delta_t*np.diag((self.M**(-1)))))
        self.state_x_transfer4 = np.hstack((np.zeros((self.dim_action,self.dim_action),dtype = np.float32),
                                            -self.delta_t*np.diag((self.M**(-1)))))

    def reset(self):
        initial_state1 = np.random.uniform(0.0,0.3,(1,self.dim_action))
        initial_state2 = np.random.uniform(-0.03,0.03,(1,self.dim_action))
        s_concate = np.hstack((initial_state1,initial_state2)).astype(np.float32)
        self.state_x  =  s_concate
        self.state_s  =  np.zeros((1,self.dim_action),dtype = np.float32)
        return self.state_x

In [ ]:
# Simulation data load from IEEE 39-bus system
data = loadmat('data/IEEE_39bus_Kron.mat')

K_EN=data['Kron_39bus']['K']
K_EN=np.asarray(K_EN, dtype=np.float32)

H=data['Kron_39bus']['H']
H=np.asarray(H, dtype=np.float32)

Damp=data['Kron_39bus']['D']
Damp=np.asarray(Damp, dtype=np.float32)

omega_R=data['Kron_39bus']['omega_R']

A_EN=data['Kron_39bus']['A']
A_EN=np.asarray(A_EN, dtype=np.float32)

gamma=data['Kron_39bus']['gamma']
gamma=np.asarray(gamma, dtype=np.float32)

In [ ]:
dim_action = 10 #dimension of action space
dim_state = 2*dim_action #dimension of state space
G_comm = nx.random_regular_graph(3, dim_action, seed=4)
print(nx.is_connected(G_comm))
incidence_comm = scipy.sparse.csr_matrix.toarray(nx.incidence_matrix(G_comm, oriented=True))

In [ ]:
import pickle
f = open('data/Comm_power.pckl', 'rb')
[G_comm,  incidence_comm]= pickle.load(f)
f.close()

In [ ]:
delta_t=0.01
M=H.reshape(dim_action)*2/omega_R*2*np.pi
D=np.zeros(dim_action,dtype=np.float32)
D[0]=2*590/100
D[1:8]=2*865/100
D[8:10]=2*911/100
D=D/omega_R*2*np.pi
F=K_EN
Penalty_action=0.01*0.2
Pm=np.array([[-0.19983394, -0.25653884, -0.25191885, -0.10242008, -0.34510365,
         0.23206371,  0.4404325 ,  0.5896664 ,  0.26257738, -0.36892462]],dtype=np.float32)

max_action=np.array([[0.19606592, 0.2190382 , 0.22375287, 0.0975513 , 0.29071101,
        0.22091283, 0.38759459, 0.56512538, 0.24151538, 0.29821917]],dtype=np.float32)*5
equilibrium_init=np.array([[ -0.05420687, -0.07780334, -0.07351729, -0.05827823, -0.09359571,
        -0.02447385, -0.00783582,  0.00259523, -0.0162409 , -0.06477749,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.       ]],dtype=np.float32)
coef_cost = np.array([0.94162537, 1.15464676, 0.55487802, 0.71176656, 1.27476119,
       1.02529959, 1.2946042 , 1.01800112, 0.99465694, 0.73467357], dtype=np.float32)
# np.random.uniform(0.5,1.5,(dim_action))
env = Frequency(Pm,M,M,D,F,delta_t,max_action,dim_action,Penalty_action, coef_cost, incidence_comm)

In [ ]:
# RNN Cell to integrate state transition dynamics
global percentage
percentage=np.random.choice([0.3,1.0,5.0], 1, p=[1/3, 1/3, 1/3])
class MinimalRNNCell(keras.layers.Layer):
    def __init__(self, units, action_units_node_p, action_units_node_i, internal_units, env, batchsize, percentage=1.0, fixed= True, env2=None, env3=None, **kwargs):
        self.units = units
        self.state_size = units
        self.action_units_node_p = action_units_node_p
        self.action_units_node_i = action_units_node_i  
        self.internal_units = internal_units
        self.batchsize = batchsize
        self.percentage = percentage
        self.fixed = fixed
        self.delta_t = tf.constant(env.delta_t,dtype = tf.float32)
        self.state_x_transfer1 = tf.constant(env.state_x_transfer1,dtype = tf.float32)
        self.state_x_transferF = tf.constant(env.state_x_transferF,dtype = tf.float32)
        self.state_x_transfer2 = tf.constant(env.state_x_transfer2,dtype = tf.float32)
        self.state_x_transfer3 = tf.constant(env.state_x_transfer3,dtype = tf.float32)
        self.state_x_transfer4 = tf.constant(env.state_x_transfer4,dtype = tf.float32)
        self.state_x_transfer3_Pm = tf.constant(env.state_x_transfer3_Pm,dtype = tf.float32)

        if not (env2==None):
            self.state_x_transfer1_2 = tf.constant(env2.state_x_transfer1,dtype = tf.float32)
            self.state_x_transferF_2 = tf.constant(env2.state_x_transferF,dtype = tf.float32)
            self.state_x_transfer2_2 = tf.constant(env2.state_x_transfer2,dtype = tf.float32)
            self.state_x_transfer3_2 = tf.constant(env2.state_x_transfer3,dtype = tf.float32)
            self.state_x_transfer4_2 = tf.constant(env2.state_x_transfer4,dtype = tf.float32)
            self.state_x_transfer3_Pm_2 = tf.constant(env2.state_x_transfer3_Pm,dtype = tf.float32)
        if not (env3==None):
            self.state_x_transfer1_3 = tf.constant(env3.state_x_transfer1,dtype = tf.float32)
            self.state_x_transferF_3 = tf.constant(env3.state_x_transferF,dtype = tf.float32)
            self.state_x_transfer2_3 = tf.constant(env3.state_x_transfer2,dtype = tf.float32)
            self.state_x_transfer3_3 = tf.constant(env3.state_x_transfer3,dtype = tf.float32)
            self.state_x_transfer4_3 = tf.constant(env3.state_x_transfer4,dtype = tf.float32)
            self.state_x_transfer3_Pm_3 = tf.constant(env3.state_x_transfer3_Pm,dtype = tf.float32)

        self.select_add_w = tf.constant(env.select_add_w,dtype = tf.float32)
        self.select_w = tf.constant(env.select_w,dtype = tf.float32)
        self.select_delta = tf.constant(env.select_delta,dtype = tf.float32)
        self.max_action = tf.constant(env.max_action,dtype = tf.float32)
        self.diag_c = tf.constant(env.diag_c, dtype = tf.float32)
        self.incidence_communication = tf.constant(env.incidence_communication,dtype = tf.float32)
        self.matrix_grad_action =  tf.constant(env.incidence_communication@env.incidence_communication.T@env.diag_c*0.05,dtype = tf.float32)

        # Matrix implementation of recover parameters in the Stacked-ReLU Neural Newrok from ancilarily variables
        self.w_recover = tf.constant(tf.linalg.band_part(-tf.ones((internal_units,internal_units)),0,1)\
                                        +2*tf.eye(internal_units),dtype = tf.float32)
        self.b_recover = tf.constant(tf.linalg.band_part(tf.ones((internal_units,internal_units)),0,-1)\
                                        -tf.eye(internal_units),dtype = tf.float32)

        ## helpful matrix for vector implementation of the proportional part
        self.Multiply_ones_node_p = tf.tile(tf.ones((action_units_node_p,action_units_node_p),dtype=np.float32)[None], [batchsize, 1, 1]) 
        self.ones_node_p = tf.ones((action_units_node_p,internal_units),dtype=tf.float32)

        ## helpful matrix for vector implementation of the integral part
        self.Multiply_ones_node_i = tf.tile(tf.ones((action_units_node_i,action_units_node_i),dtype=np.float32)[None], [batchsize, 1, 1]) 
        self.ones_node_i = tf.ones((action_units_node_i,internal_units),dtype=tf.float32)
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        # Intermediate parameters to enforce constraints of the monotone neural network 
        ## for proportional part
        self.w_plus_temp0_node_p =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units),
            initializer=tf.constant_initializer(1.0),
            trainable=True,
            name='w_plus_temp_node_p')

        self.b_plus_temp0_node_p = self.add_weight(
            shape=(self.action_units_node_p,self.internal_units),
            initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=0.2),
            trainable=True,
            constraint=tf.keras.constraints.MaxNorm(0.2),
            name='b_plus_temp_node_p')
        self.w_minus_temp0_node_p =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units),
            initializer=tf.constant_initializer(1.0),
            trainable=True,
            name='w_minus_temp_node_p')

        self.b_minus_temp0_node_p = self.add_weight(
            shape=(self.action_units_node_p,self.internal_units),
            initializer=tf.keras.initializers.RandomUniform(minval=0, maxval=0.2),
            trainable=True,
            constraint=tf.keras.constraints.MaxNorm(0.2),
            name='b_minus_temp_node_p')
        
        
        ## for integral part
        self.k_integral = self.add_weight(
            shape=(1,),
            initializer=tf.constant_initializer(6.0), #2.64
            trainable=True,
            name='k_integral'
        )
        
        self.built = True

    def call(self, inputs, states):  
        # percentage=np.random.choice([0.2,0.6,1.0], 1, p=[0.3, 0.4, 0.3])  
        self.percentage = percentage
        prev_state = states[0] # Size: Batch_size*state_size  300*30
        prev_state_x = prev_state[:,0:self.action_units_node_p*2]
        prev_state_s =  prev_state[:,self.action_units_node_p*2:self.action_units_node_p*3]
        #########   control action parameterization ##########
        # Monotone neural network by stacking relu function
        ## for proportional part
        w_plus_temp_node_p = tf.math.square(self.w_plus_temp0_node_p)
        b_plus_temp_node_p = tf.math.square(self.b_plus_temp0_node_p)
        w_minus_temp_node_p = tf.math.square(self.w_minus_temp0_node_p)
        b_minus_temp_node_p = tf.math.square(self.b_minus_temp0_node_p)
        w_plus_node_p = K.dot(w_plus_temp_node_p,self.w_recover)
        b_plus_node_p = K.dot(-b_plus_temp_node_p,self.b_recover)
        w_minus_node_p = K.dot(-w_minus_temp_node_p,self.w_recover)
        b_minus_node_p = K.dot(-b_minus_temp_node_p,self.b_recover)

        nonlinear_plus_node_p = K.sum(K.relu(K.dot(tf.linalg.diag( K.dot(prev_state_x,self.select_w)),self.ones_node_p)+b_plus_node_p)\
                        *w_plus_node_p,axis=2)  
        nonlinear_minus_node_p = K.sum(K.relu(-K.dot(tf.linalg.diag(K.dot(prev_state_x,self.select_w)),self.ones_node_p)+b_minus_node_p)\
                        *w_minus_node_p,axis=2)  

        action_nonconstrain0_node_p = (nonlinear_plus_node_p+nonlinear_minus_node_p)
        action_node_p = action_nonconstrain0_node_p


        ## for integral part
        k_int = tf.math.abs(self.k_integral)
        # # K_int_matrix = tf.linalg.diag(k_int)
        # # action_node_i = K.dot(prev_state_s, K_int_matrix)
        
        action_node_i = prev_state_s * k_int

        ## PI control law            
        action_nonconstrain = action_node_i + action_node_p
        ## make the action between higher and lower bound
        action =  self.max_action-K.relu(self.max_action-action_nonconstrain)+K.relu(-self.max_action-action_nonconstrain)


        #########   calculate state transition functions  #######      
        # calculate state on s
        # grad_action = tf.math.pow(action_node_i,3)@self.diag_c  # quardratic
        grad_action = action_node_i@self.diag_c
        dot_s = K.dot(prev_state_x,self.select_w)- K.dot(grad_action, self.matrix_grad_action)

        # integrate in the state transition dynamics
        if self.percentage == 0.3:
            if not self.fixed:
                self.percentage=np.random.choice([0.3,1.0], 1, p=[0.8, 0.2])
            new_state_x = K.dot(prev_state_x, self.state_x_transfer1)+\
                K.dot(K.sum(K.sin( K.dot(tf.linalg.diag(K.dot(prev_state_x, self.select_delta)),tf.ones((dim_action,dim_action),dtype = np.float32))-\
                                    tf.matmul(self.Multiply_ones_node_p,tf.linalg.diag(K.dot(prev_state_x, self.select_delta))))\
                            *self.state_x_transferF,axis = 2 )\
                                        ,self.state_x_transfer2)\
                                + self.state_x_transfer3+K.dot(action,self.state_x_transfer4)\
                                + inputs@self.state_x_transfer3_Pm
        elif self.percentage == 1.0: #0.6
            if not self.fixed:
                self.percentage=np.random.choice([0.3,1.0,5.0], 1, p=[0.1, 0.8, 0.1])
            new_state_x = K.dot(prev_state_x, self.state_x_transfer1_2)+\
                K.dot(K.sum(K.sin( K.dot(tf.linalg.diag(K.dot(prev_state_x, self.select_delta)),tf.ones((dim_action,dim_action),dtype = np.float32))-\
                                    tf.matmul(self.Multiply_ones_node_p,tf.linalg.diag(K.dot(prev_state_x, self.select_delta))))\
                            *self.state_x_transferF_2,axis = 2 )\
                                        ,self.state_x_transfer2_2)\
                                + self.state_x_transfer3_2+K.dot(action,self.state_x_transfer4_2)\
                                + inputs@self.state_x_transfer3_Pm_2
        else:   # percentage == 1.0:
            if not self.fixed:
                self.percentage=np.random.choice([1.0,5.0], 1, p=[0.2, 0.8])
            new_state_x = K.dot(prev_state_x, self.state_x_transfer1_3)+\
                K.dot(K.sum(K.sin( K.dot(tf.linalg.diag(K.dot(prev_state_x, self.select_delta)),tf.ones((dim_action,dim_action),dtype = np.float32))-\
                                    tf.matmul(self.Multiply_ones_node_p,tf.linalg.diag(K.dot(prev_state_x, self.select_delta))))\
                            *self.state_x_transferF_3,axis = 2 )\
                                        ,self.state_x_transfer2_3)\
                                + self.state_x_transfer3_3+K.dot(action,self.state_x_transfer4_3)\
                                + inputs@self.state_x_transfer3_Pm_3


        loss0 = K.dot(K.pow(new_state_x,2),self.select_add_w)
        frequency = K.dot(new_state_x,self.select_w)
        new_state_s = prev_state_s + self.delta_t*dot_s
        next_state = tf.concat([new_state_x,  new_state_s], axis = 1)        
        return [loss0, frequency, action_node_p, action_node_i, action], [next_state]

In [ ]:
# Train controller with inertia mode 5
env = Frequency(Pm, M, M * 0.3,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env2 = Frequency(Pm, M, M * 1.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env3 = Frequency(Pm, M, M * 5.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
start = time.time()

episodes  = 500 # total number of iterations to update weights
units = dim_action*3 #dimension of each state
internal_units = 20 #number of neurons in each hidden layer
T = 300  #Total time period
Batch_num = 300 # number of batch in each episodes


# RNN initialization
cell_5 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=5.0, fixed=True,env2=env2,env3=env3)
layer_5 = RNN(cell_5,return_sequences = True,stateful = True)
input_5 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_5((input_5))
model_mono_5 = tf.keras.models.Model([input_5], outputs)
model_mono_5.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

x0 = np.ones((Batch_num,T,dim_action))
y0 = model_mono_5(x0)
global_step = tf.Variable(0, trainable = False)
learning_rate_initial = 0.05
decayed_lr  = tf.keras.optimizers.schedules.ExponentialDecay(
    learning_rate_initial, 50, 0.8, staircase = True)
optimizer = tf.keras.optimizers.Adam(learning_rate = decayed_lr)
# optimizer = tf.keras.optimizers.AdamW(learning_rate = decayed_lr)

# Randomly generated traning set 
PrintUpdate = 1
num_gen_step = 3 # pickup three generators of have a step load change
Percent_step_change = 1
range_step_change = 3 # upper and lower bound of the step load change
Loss_record_mono = []

# Training
for i in range(0,episodes):
    initial_state = np.zeros((Batch_num,dim_action*2))+equilibrium_init
    Pm_change = np.zeros((Batch_num,T,dim_action))
    percentage = 5.0
    
    for gen_interupt in range(0, num_gen_step):
        idx_gen_deviation = np.random.randint(0, dim_action, Batch_num*Percent_step_change)
        idx_batch_deviation = np.random.randint(0, Batch_num, Batch_num*Percent_step_change)
        slot_start_deviation = np.random.randint(0, T, Batch_num*Percent_step_change)  
        step_change = np.random.uniform(-1,1,(Batch_num*Percent_step_change))*range_step_change    
        for t_interupt in range(0,T):
            Pm_change[idx_batch_deviation,t_interupt, idx_gen_deviation]\
                            = (slot_start_deviation>= t_interupt)*step_change
    
    layer_5.reset_states( np.hstack((initial_state, np.zeros((Batch_num,dim_action)))))
    with tf.GradientTape(persistent = True) as tape:
        [loss0,frequency,action0,actions, action] = model_mono_5(Pm_change)
        loss_action = 0.5 * K.sum(K.pow(action,2)@env.diag_c)/Batch_num/T 
        loss_freq = 10*K.sum(tf.norm(frequency, axis=2))/Batch_num/T  + 10*K.sum(K.max(K.abs(frequency),axis = 2))/Batch_num/T 
        loss = 1*loss_action + 100*loss_freq

    grads = tape.gradient(loss, model_mono_5.variables)
    optimizer.apply_gradients(zip(grads, model_mono_5.variables))  
    Loss_record_mono.append(loss)
    if i % (PrintUpdate) ==  0:
        print('episode',i, 'Loss',loss)
        print('episode',i, 'Loss_frequency',loss_freq)

end = time.time()
print(end - start) 
model_mono_5.save_weights('checkpoints/M5/weights_RNN_M5.ckpt')

In [ ]:
# Train Neural-PI-0.3
env = Frequency(Pm, M, M*0.3,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env2 = Frequency(Pm, M, M*1.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env3 = Frequency(Pm, M, M*5.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)

start = time.time()

episodes  = 500 # total number of iterations to update weights
units = dim_action*3 #dimension of each state
internal_units = 20 # number of neurons in each hidden layer
T = 300  #Total time period
Batch_num = 300 # number of batch in each episodes


# RNN initialization
cell_03 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=0.3, fixed=True,env2=env2,env3=env3)
layer_03 = RNN(cell_03,return_sequences = True,stateful = True)
input_03 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_03((input_03))
model_mono_03 = tf.keras.models.Model([input_03], outputs)
model_mono_03.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
# model_mono_05.load_weights('/home/jason/Documents/research/switching/Neural-PI/checkpoints/M02/weights_RNN_M02.ckpt')

x0 = np.ones((Batch_num,T,dim_action))
y0 = model_mono_03(x0)
global_step = tf.Variable(0, trainable = False)
learning_rate_initial = 0.05
decayed_lr  = tf.keras.optimizers.schedules.ExponentialDecay(
    learning_rate_initial, 50, 0.7, staircase = True)
optimizer = tf.keras.optimizers.Adam(learning_rate = decayed_lr)

# Randomly generated traning set 
PrintUpdate = 1
num_gen_step = 3 # pickup three generators of have a step load change
Percent_step_change = 1
range_step_change = 3 # upper and lower bound of the step load change
Loss_record_mono = []

# Training
for i in range(0,episodes):
    initial_state = np.zeros((Batch_num,dim_action*2))+equilibrium_init
    Pm_change = np.zeros((Batch_num,T,dim_action))
    percentage=0.3
    
    for gen_interupt in range(0, num_gen_step):
        idx_gen_deviation = np.random.randint(0, dim_action, Batch_num*Percent_step_change)
        idx_batch_deviation = np.random.randint(0, Batch_num, Batch_num*Percent_step_change)
        slot_start_deviation = np.random.randint(0, T, Batch_num*Percent_step_change)  
        step_change = np.random.uniform(-1,1,(Batch_num*Percent_step_change))*range_step_change    
        for t_interupt in range(0,T):
            Pm_change[idx_batch_deviation,t_interupt, idx_gen_deviation]\
                            = (slot_start_deviation>= t_interupt)*step_change
    
    layer_03.reset_states( np.hstack((initial_state, np.zeros((Batch_num,dim_action)))))
    with tf.GradientTape(persistent = True) as tape:
        [loss0,frequency,action0,actions, action] = model_mono_03(Pm_change)
        loss_action = 0.5*K.sum(K.pow(action,2)@env.diag_c)/Batch_num/T 
        loss_freq = 10*K.sum(tf.norm(frequency, axis=2))/Batch_num/T + 10*K.sum(K.max(K.abs(frequency),axis = 2))/Batch_num/T 
        loss = 1.1*loss_action + 100*loss_freq

    grads = tape.gradient(loss, model_mono_03.variables)
    optimizer.apply_gradients(zip(grads, model_mono_03.variables))  
    Loss_record_mono.append(loss)
    if i % (PrintUpdate) ==  0:
        print('episode',i, 'Loss',loss)
        print('episode',i, 'Loss_frequency',loss_freq)

end = time.time()
print(end - start) 
model_mono_03.save_weights('checkpoints/M02/weights_RNN_M02.ckpt')

In [ ]:
# Train Neural-PI-1
env = Frequency(Pm, M, M*0.3,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env2 = Frequency(Pm, M, M*1.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env3 = Frequency(Pm, M, M*5.0,D,F,delta_t,max_action*10,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)

start = time.time()

episodes  = 500 # total number of iterations to update weights
units = dim_action*3 #dimension of each state
internal_units = 20 # number of neurons in each hidden layer
T = 300  #Total time period
Batch_num = 300 # number of batch in each episodes


# RNN initialization
cell_1 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=1.0, fixed=True,env2=env2,env3=env3)
layer_1 = RNN(cell_1,return_sequences = True,stateful = True)
input_1 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_1((input_1))
model_mono_1 = tf.keras.models.Model([input_1], outputs)
model_mono_1.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

x0 = np.ones((Batch_num,T,dim_action))
y0 = model_mono_1(x0)
global_step = tf.Variable(0, trainable = False)
learning_rate_initial = 0.05
decayed_lr  = tf.keras.optimizers.schedules.ExponentialDecay(
    learning_rate_initial, 50, 0.70, staircase = True)
optimizer = tf.keras.optimizers.Adam(learning_rate = decayed_lr)

# Randomly generated traning set 
PrintUpdate = 1
num_gen_step = 3 # pickup three generators of have a step load change
Percent_step_change = 1
range_step_change = 3 # upper and lower bound of the step load change
Loss_record_mono = []

# Training
for i in range(0,episodes):
    # env.switch(new_percentage)
    initial_state = np.zeros((Batch_num,dim_action*2))+equilibrium_init
    Pm_change = np.zeros((Batch_num,T,dim_action))
    # percentage=np.random.choice([0.3,1.0], 1, p=[0.01, 0.99])
    percentage=1.0
    
    for gen_interupt in range(0, num_gen_step):
        idx_gen_deviation = np.random.randint(0, dim_action, Batch_num*Percent_step_change)
        idx_batch_deviation = np.random.randint(0, Batch_num, Batch_num*Percent_step_change)
        slot_start_deviation = np.random.randint(0, T, Batch_num*Percent_step_change)  
        step_change = np.random.uniform(-1,1,(Batch_num*Percent_step_change))*range_step_change    
        for t_interupt in range(0,T):
            Pm_change[idx_batch_deviation,t_interupt, idx_gen_deviation]\
                            = (slot_start_deviation>= t_interupt)*step_change
    
    layer_1.reset_states( np.hstack((initial_state, np.zeros((Batch_num,dim_action)))))
    with tf.GradientTape(persistent = True) as tape:
        [loss0,frequency,action0,actions, action] = model_mono_1(Pm_change)
        loss_action = 0.5*K.sum(K.pow(action,2)@env.diag_c)/Batch_num /T 
        # loss_freq = 1*K.sum(K.max(K.abs(frequency),axis = 1))/Batch_num + 0.05*K.sum(K.abs(frequency))/Batch_num
        loss_freq = 10*K.sum(tf.norm(frequency, axis=2))/Batch_num /T + 10*K.sum(K.max(K.abs(frequency),axis = 2))/Batch_num/T 
        loss = 1*loss_action + 100*loss_freq

    grads = tape.gradient(loss, model_mono_1.variables)
    optimizer.apply_gradients(zip(grads, model_mono_1.variables))  
    Loss_record_mono.append(loss)
    if i % (PrintUpdate) ==  0:
        print('episode',i, 'Loss',loss)
        print('episode',i, 'Loss_frequency',loss_freq)

end = time.time()
print(end - start) 
model_mono_1.save_weights('checkpoints/M1/weights_RNN_M1.ckpt')

# Test the trained model

In [ ]:
def Action_P(state, model, cell, env):
    
    w_plus=K.dot(tf.math.square(model.variables[0]),cell.w_recover)
    b_plus=K.dot(-tf.math.square(model.variables[1]),cell.b_recover)
    w_minus=K.dot(-tf.math.square(model.variables[2]),cell.w_recover)
    b_minus=K.dot(-tf.math.square(model.variables[3]),cell.b_recover)
    nonlinear_plus=K.sum(K.relu(K.dot(tf.linalg.diag(state@env.select_w),cell.ones_node_p)+b_plus)\
                    *w_plus,axis=2)  
    nonlinear_minus=K.sum(K.relu(-K.dot(tf.linalg.diag(state@env.select_w),cell.ones_node_p)+b_minus)\
                    *w_minus,axis=2)  
    action_nonconstrain0= nonlinear_plus+nonlinear_minus


    return action_nonconstrain0

def Action_I(state_s, model, cell, env):
    # state_s = tf.convert_to_tensor(state_s)
    # k_int = tf.math.abs(model.variables[4].numpy())
    k_int = 9 # this is a rounded average of the trained k for the integral controller, which is shared by different base controllers
    action_node_i = k_int*state_s
    return action_node_i

def Action(state_x, state_s, model, cell, env):
    action_nonconstrain0 = Action_P(state_x, model, cell, env)
    action_nonconstrain1 = Action_I(state_s, model, cell, env)
    action_nonconstrain =  action_nonconstrain0 + action_nonconstrain1
    action=env.max_action-K.relu(env.max_action-action_nonconstrain)+K.relu(-env.max_action-action_nonconstrain)
    return action, action_nonconstrain0, action_nonconstrain1

# Load Models

In [ ]:
env = Frequency(Pm, M, M*0.3, D,F,delta_t,max_action,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env2 = Frequency(Pm, M, M*1.0, D,F,delta_t,max_action,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)
env3 = Frequency(Pm, M, M*5.0, D,F,delta_t,max_action,dim_action,Penalty_action*3, coef_cost, incidence_comm, gamma=5)

episodes  = 1500 # total number of iterations to update weights
units = dim_action*3 #dimension of each state
internal_units = 20 # number of neurons in each hidden layer
T = 400  #Total time period
Batch_num = 300 # number of batch in each episodes

cell_03 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=0.3, fixed=True,env2=env2,env3=env3)
layer_03 = RNN(cell_03,return_sequences = True,stateful = True)
input_03 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_03((input_03))
model_mono_03 = tf.keras.models.Model([input_03], outputs)
model_mono_03.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
model_mono_03.load_weights('checkpoints/M02/weights_RNN_M02.ckpt')

In [ ]:
cell_1 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=1.0, fixed=True,env2=env2,env3=env3)
layer_1 = RNN(cell_1,return_sequences = True,stateful = True)
input_1 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_1((input_1))
model_mono_1 = tf.keras.models.Model([input_1], outputs)
model_mono_1.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
model_mono_1.load_weights('/checkpoints/M1/weights_RNN_M1.ckpt')

In [ ]:
cell_5 = MinimalRNNCell(units,dim_action, dim_action, internal_units,env,Batch_num, percentage=0.3, fixed=True,env2=env2,env3=env3)
layer_5 = RNN(cell_5,return_sequences = True,stateful = True)
input_5 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
outputs = layer_5((input_5))
model_mono_5 = tf.keras.models.Model([input_5], outputs)
model_mono_5.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
model_mono_5.load_weights('checkpoints/M5/weights_RNN_M5.ckpt')

# Test in time-invariant inertias

dist = np.random.uniform(-1,1,(100,(Pm_init).shape[1])) # generate disturbance for 100 trajectories

In [ ]:
# Record for fixed single environment
controller_set = [layer_03, layer_1, layer_5]
cell_set = [cell_03, cell_1, cell_5]
env = Frequency(Pm,M, M*5,D,F,delta_t,max_action, dim_action,Penalty_action*3, coef_cost, incidence_comm,gamma=5)
SimulationLength = 300

id = 2#controller id
loss_list=[] 
action_loss_list = []
freq_loss_list = []
for j in range(100):
    Record_u = []
    record_freq = []
    init_state=equilibrium_init

    # init_state=env.reset()
    x=init_state.copy().astype(np.float32)
    env.set_state(x)
    Loss_RNN=0
    Loss_RNN_discounted=0
    Pm_init=Pm.copy()
    Pm1=Pm_init.copy().astype(np.float32)
    Pm2=(Pm_init.copy()).astype(np.float32)

    Pm2 = Pm2 + dist[j]
    action_s = np.zeros((1,dim_action),dtype=np.float32)
    record_grad_ui = []
    Trajectory_s = []
    Trajectory_s.append(np.squeeze(env.state_s))
    for i in range(SimulationLength):
        if i<int(10) :
            Pm_change=Pm1.copy()
        if i>=int(10) and i<int(400):
            Pm_change=Pm2.copy()

        u, up, ui = Action(x, action_s, controller_set[id], cell_set[id], env)

        action_s, next_x, r= env.step_PI_CommEdge(up,ui,Pm_change)
        freq = np.dot(next_x, env.select_w)
        record_freq.append(freq)

        Loss_RNN_discounted+=r
        Loss_RNN+=r
        x=next_x
        Record_u.append(np.squeeze(u))
        

    record_freq = np.squeeze(np.asarray(record_freq))
    action_array = np.asarray(Record_u)
    loss_action = 0.5*np.mean(np.sum(np.power(action_array,2)@env.diag_c,axis=1))
    # loss_freq = np.sum(np.max(np.abs(record_freq)))/record_freq.shape[0] + 2*np.sum(np.abs(record_freq))/record_freq.shape[0]
    loss_freq = 10*np.mean(np.linalg.norm(record_freq,axis=1)) + 10*np.mean(np.max(np.abs(record_freq),axis=1))
    loss = 1*loss_action + 100*loss_freq
    loss_list.append(loss.copy())
    action_loss_list.append(loss_action.copy())
    freq_loss_list.append(loss_freq.copy()*100)
print(np.mean(loss_list),np.std(loss_list))
print(np.mean(action_loss_list),np.std(action_loss_list))
print(np.mean(freq_loss_list),np.std(freq_loss_list))

# controller 0.3    1.0   5.0 
# 0.3 M:     22.50 (8.25),  335.07 (2.84), 488.07 (3.03)
# 1.0 M:     22.74 (8.37),  11.23 (2.40), 13.10 （4.47）
# 5.0 M:     23.81 (8.35),  11.24（2.34）, 10.52 (2.06) 


# Long horizon, two disturbances

In [ ]:
controller_set = [layer_05, layer_1, layer_03]
cell_set = [cell_05, cell_1, cell_03]
id = 0

action_loss_list = []
freq_loss_list = []
total_loss_list = []
transient_action_cost_list = []
transient_freq_cost_list = []
transient_cost_list = []

for i in range(100):
    percentage_plot = np.random.choice([0.3,1.0,5.0], 1, p=[0.1, 0.45, 0.45])[0]
    env = Frequency(Pm, M, M*percentage_plot,D,F,delta_t,max_action, dim_action,Penalty_action*3, coef_cost, incidence_comm,gamma=5)
    Trajectory_RNN=[] 
    Record_u=[]
    Record_up=[]
    Record_ui=[]
    record_freq = []
    record_freq_norm =[]
    record_grad_ui = []
    Trajectory_s = []
    mode_traj = []

    init_state=equilibrium_init
    transient_list = []
    transient_action_list = []

    # init_state=env.reset()
    x=init_state.copy().astype(np.float32)
    env.set_state(x)
    Trajectory_RNN.append(x)
    SimulationLength=2000
    Pm_init=Pm.copy()
    action_s = np.zeros((1,dim_action),dtype=np.float32)
    transient_start = False
    
    mode_traj.append(percentage_plot)
    Pm_change=Pm_init.copy()
    for j in range(SimulationLength):
        if j==10:
            Pm_change=Pm_init.copy() + np.random.uniform(-1,1,((Pm_init).shape[1]))
            transient_start = True
            transient_time = 0
        elif j ==700:
            Pm_change=Pm_init.copy() + np.random.uniform(-1,1,((Pm_init).shape[1]))
            transient_start = True
            transient_time = 0

        if (j+1)%500 == 0:
            if percentage_plot == 0.3:
                percentage_plot = np.random.choice([0.3,1.0], 1, p=[0.5, 0.5])[0]
            elif percentage_plot == 1.0:
                percentage_plot = np.random.choice([0.3,1.0,5.0], 1, p=[0.3, 0.4, 0.3])[0]
            else:
                percentage_plot = np.random.choice([1.0,5.0], 1, p=[0.5, 0.5])[0]
            env.switch(percentage_plot)
            mode_traj.append(percentage_plot)

        u, up, ui = Action(x, action_s, controller_set[id], cell_set[id], env)

        action_s, next_x, r= env.step_PI_CommEdge(up,ui,Pm_change)
        freq = np.dot(next_x, env.select_w)
        if transient_start:
            transient_list.append(freq)
            transient_action_list.append(u)
            transient_time +=1
            if transient_time==300:
                transient_start = False
        record_freq.append(freq)
        record_freq_norm.append(np.linalg.norm(freq))

        x=next_x
        Trajectory_RNN.append(x)
        Record_u.append(np.squeeze(u))
        Record_up.append(np.squeeze(up))
        Record_ui.append(np.squeeze(ui))
        record_grad_ui.append(np.squeeze(env.calc_grad_action(ui)[1]))
        Trajectory_s.append(np.squeeze(env.state_s))

    record_freq = np.squeeze(np.asarray(record_freq))
    action_array = np.asarray(Record_u)

    loss_action = 0.5*np.mean(np.sum(np.power(action_array,2)@env.diag_c,axis=1))
    # loss_freq = np.sum(np.max(np.abs(record_freq)))/record_freq.shape[0] + 2*np.sum(np.abs(record_freq))/record_freq.shape[0]
    loss_freq = 10*np.mean(np.linalg.norm(record_freq,axis=1))+ 10*np.mean(np.max(np.abs(record_freq),axis=1))
    loss = 1*loss_action + 100*loss_freq
    action_loss_list.append(loss_action)
    freq_loss_list.append(100*loss_freq)
    total_loss_list.append(loss)
    
    transient_freq = 10*np.mean(np.linalg.norm(np.squeeze(transient_list),axis=1)) + 10*np.mean(np.max(np.abs(transient_list),axis=1))
    transient_act = np.squeeze(np.asarray(transient_action_list))
    transient_act_cost = 0.5*np.mean(np.sum(np.power(transient_act,2)@env.diag_c,axis=1))
    trans_cost = transient_act_cost + 100*transient_freq
    transient_cost_list.append(trans_cost)
    transient_action_cost_list.append(transient_act_cost)
    transient_freq_cost_list.append(transient_freq*100)

    # print(loss,loss_action,5*loss_freq)
print('total cost:', np.mean(total_loss_list), np.std(total_loss_list))
print('freq: ',np.mean(freq_loss_list),np.std(freq_loss_list))
print('action: ',np.mean(action_loss_list),np.std(action_loss_list))

print('***********************Transient**************************')
print('transient total: ',np.mean(transient_cost_list),np.std(transient_cost_list))
print('transient freq: ', np.mean(transient_freq_cost_list),np.std(transient_freq_cost_list))
print('transient action: ', np.mean(transient_action_cost_list), np.std(transient_action_cost_list))


# Exp-ISS with event-triggering

In [ ]:
class exp3_ISS_algm():
    def __init__(self, tau, controller_set, cell_set,eta):
        self.controller_set = controller_set
        self.cell_set = cell_set
        self.tau = tau
        self.greedy = 0.00
        self.controller_num = len(self.controller_set)
        self.p = np.ones(self.controller_num)/self.controller_num
        self.G = np.zeros(self.controller_num)
        self.g = np.zeros(self.controller_num)
        self.g_hat = np.zeros(self.controller_num)
        self.controller_idx_set = [i for i in range(self.controller_num)]
        self.batch_cost_list = []
        self.freq_before_switch = 0
        self.current_controller_id = self.sample()
        self.eta = eta

        self.detect_step = 0
        self.detect_start=False
        self.detect_finished = 0
        self.detect_version_record = False
        
        self.old_controller_id = -1
    def current_p(self):
        return self.p
                
        
    def update(self,cost_batch,controller_idx,eta=0.5):
        epsilon = 0.001
        for cidx in self.controller_idx_set:
            if cidx == controller_idx:
                self.g_hat[cidx] = cost_batch/(self.p[cidx]+epsilon)
            self.G[cidx] = self.G[cidx] + self.g_hat[cidx]
        tmp = 0
        for cidx in self.controller_idx_set:
            tmp += np.exp(-self.eta*self.G[cidx])
        for cidx in self.controller_idx_set:
            self.p[cidx] = (np.exp(-self.eta*self.G[cidx]))/ (tmp)
        self.g_hat = np.zeros(self.controller_num)
        return self.p
    
    def detect(self, cost, freq, step):
        detech_length = 50
        cold_period = 300
        detect_version = (np.max(np.abs(freq))>0.01)     
        if detect_version and (not self.detect_start):
            self.detect_start=True
        if self.detect_start:
            if self.detect_step<detech_length:
                p = self.collect(cost, freq)
            self.detect_step += 1
            if self.detect_step>detech_length:
                self.current_controller_id = np.argmax(self.p)
                if (np.max(np.abs(freq))>0.15):
                    self.detect_step=0
                    self.detect_start=False
            if self.detect_step == (detech_length+cold_period):
                self.detect_step=0
                self.detect_start=False
        return self.p
    
    def sample(self):
        controller_idx = np.random.choice(self.controller_idx_set, p=self.p)
        return controller_idx
    
    def collect(self, cost, freq):
        self.batch_cost_list.append(cost.copy())
        if len(self.batch_cost_list)==self.tau:
            cost_batch = np.sum(self.batch_cost_list)            
            self.old_controller_id = self.current_controller_id.copy()
            new_p = self.update(cost_batch,self.current_controller_id)
            self.batch_cost_list = []
            self.freq_before_switch = freq.copy()
            self.current_controller_id = self.sample()
        else:
            new_p = self.p
        return new_p
    
    def get_action(self, x, action_s, env):
        u, up, ui = Action(x, action_s, self.controller_set[self.current_controller_id],\
                            self.cell_set[self.current_controller_id], env)
        return u, up, ui


In [ ]:
action_loss_list = []
freq_loss_list = []
total_loss_list = []
transient_action_cost_list = []
transient_freq_cost_list = []
transient_cost_list = []
# controller_set = [layer_03, layer, layer_5]
# cell_set = [cell_03, cell, cell_5]
controller_set = [layer_05, layer_1, layer_03]
cell_set = [cell_05, cell_1, cell_03]
p_traj_list = []
mode_predefined = [5.0,1.0,1.0,0.3,0.3]
tau = 5
for run_num in range(100):
  switching_agent = exp3_ISS_algm(tau,controller_set,cell_set,eta=0.005)
  p = switching_agent.current_p()
  percentage_plot = np.random.choice([0.3,1.0,5.0], 1, p=[0.1, 0.45, 0.45])[0]
  # percentage_plot=mode_predefined[0]
  env = Frequency(Pm,M,M*percentage_plot,D,F,delta_t,max_action, dim_action,Penalty_action*3, coef_cost, incidence_comm,gamma=5)
  Trajectory_RNN=[] 
  Record_u=[]
  Record_up=[]
  Record_ui=[]
  record_freq = []
  record_freq_norm = []
  record_grad_ui = []
  Trajectory_s = []
  mode_traj = []
  transient_list = []
  transient_action_list = []
  transient_start = False
  p_traj = []
  mode_traj.append(percentage_plot)

  init_state=equilibrium_init

  # init_state=env.reset()
  x=init_state.copy().astype(np.float32)

  env.set_state(x)
  Trajectory_RNN.append(x)
  SimulationLength=2000
  Pm_init=Pm.copy()
  # print(dist1.shape)
  action_s = np.zeros((1,dim_action),dtype=np.float32)

  Pm_change = Pm_init.copy() 
  for i in range(SimulationLength):
      if i==10:
        Pm_change=Pm_init.copy() + np.random.uniform(-1,1,((Pm_init).shape[1]))
        transient_start = True
        transient_time = 0
      elif i == 700:
        Pm_change=Pm_init.copy() + np.random.uniform(-1,1,((Pm_init).shape[1]))
        transient_start = True
        transient_time = 0

      if (i+1)%500 == 0:
        if percentage_plot == 0.3:
            percentage_plot = np.random.choice([0.3,1.0], 1, p=[0.5, 0.5])[0]
        elif percentage_plot == 1.0:
            percentage_plot = np.random.choice([0.3,1.0,5.0], 1, p=[0.3, 0.4, 0.3])[0]
        else:
            percentage_plot = np.random.choice([1.0,5.0], 1, p=[0.5, 0.5])[0]
        env.switch(percentage_plot)
        # percentage_plot = 0.3
        # percentage_plot= mode_predefined[(i+1)//500]
        mode_traj.append(percentage_plot)

      u, up, ui = switching_agent.get_action(x, action_s,env)

      action_s, next_x, r= env.step_PI_CommEdge(up,ui,Pm_change)
      
      p_traj.append(p.copy())
      # print(next_x.shape)
      freq = np.dot(next_x, env.select_w)
      # p=switching_agent.collect(r,freq)
         
      p = switching_agent.detect(r,freq,i)
      
      record_freq.append(freq)
      if transient_start:
        transient_list.append(freq)
        transient_action_list.append(u)
        transient_time += 1
        if transient_time == 300:
            transient_start=False
      record_freq_norm.append(np.linalg.norm(freq))


      x=next_x
      Trajectory_RNN.append(x)
      Record_u.append(np.squeeze(u))
      Record_up.append(np.squeeze(up))
      Record_ui.append(np.squeeze(ui))
      
      record_grad_ui.append(np.squeeze(env.calc_grad_action(ui)[1]))
      Trajectory_s.append(np.squeeze(env.state_s))

  record_freq = np.squeeze(np.asarray(record_freq))
  action_array = np.asarray(Record_u)
  transient_list = np.squeeze(transient_list)
  p_traj_list.append(p_traj)


  loss_action = 0.5*np.mean(np.sum(np.power(action_array,2)@env.diag_c,axis=1))
  # loss_freq = np.sum(np.max(np.abs(record_freq)))/record_freq.shape[0] + 2*np.sum(np.abs(record_freq))/record_freq.shape[0]
  loss_freq = 10*np.mean(np.linalg.norm(record_freq,axis=1))+ 10*np.mean(np.max(np.abs(record_freq),axis=1))
  loss = 1*loss_action + 100*loss_freq
  action_loss_list.append(loss_action)
  freq_loss_list.append(100*loss_freq)
  total_loss_list.append(loss)
  
  transient_freq = 10*np.mean(np.linalg.norm(np.squeeze(transient_list),axis=1)) + 10*np.mean(np.max(np.abs(transient_list),axis=1))
  transient_act = np.squeeze(np.asarray(transient_action_list))
  transient_act_cost = 0.5*np.mean(np.sum(np.power(transient_act,2)@env.diag_c,axis=1))
  trans_cost = transient_act_cost + 100*transient_freq
  transient_cost_list.append(trans_cost)
  transient_action_cost_list.append(transient_act_cost)
  transient_freq_cost_list.append(transient_freq*100)

    # print(loss,loss_action,5*loss_freq)
print('total cost:', np.mean(total_loss_list), np.std(total_loss_list))
print('freq: ',np.mean(freq_loss_list),np.std(freq_loss_list))
print('action: ',np.mean(action_loss_list),np.std(action_loss_list))

print('***********************Transient**************************')
print('transient total: ',np.mean(transient_cost_list),np.std(transient_cost_list))
print('transient freq: ', np.mean(transient_freq_cost_list),np.std(transient_freq_cost_list))
print('transient action: ', np.mean(transient_action_cost_list), np.std(transient_action_cost_list))